<a href="https://colab.research.google.com/github/NK1502/Projects/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

# Load the training dataset
train_df = pd.read_csv('train_dataset.csv')

# Load the test dataset
test_df = pd.read_csv('test_dataset.csv')

# Display the first few rows of the training data
print("Training Data:")
print(train_df.head())

# Display the first few rows of the test data
print("\nTest Data:")
print(test_df.head())

Training Data:
                                     url  label                  source  \
0     irs-profilepaymentservice.com/home      1               phishtank   
1                       cpuggsukabumi.id      0        majestic_million   
2    members.tripod.com/~don_rc/ring.htm      0  data_clean_test_mendel   
3  optuswebmailadminprovider.weebly.com/      1               phishtank   
4                           topoz.com.pl      0            dmoz_harvard   

   url_has_login  url_has_client  url_has_server  url_has_admin  url_has_ip  \
0              0               0               0              0           0   
1              0               0               0              0           0   
2              0               0               0              0           0   
3              0               0               0              1           0   
4              0               0               0              0           0   

   url_isshorted  url_len  ...  pdomain_count_hyphen  pdoma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [6]:
# Separate features (X) and target (y) for training data
X_train = train_df.drop('label', axis=1)  # Features
y_train = train_df['label']  # Target variable

# Separate features (X) and target (y) for test data
X_test = test_df.drop('label', axis=1)  # Features
y_test = test_df['label']  # Target variable

In [7]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Apply label encoding to categorical columns
for column in X_train.select_dtypes(include=['object']).columns:
    X_train[column] = le.fit_transform(X_train[column])
    X_test[column] = le.fit_transform(X_test[column])

print(X_test.head())
print(X_train.head())

    url  source  url_has_login  url_has_client  url_has_server  url_has_admin  \
0  2992       6              0               0               0              0   
1  2045       8              0               0               0              0   
2  3602       3              0               0               0              0   
3  1167       5              0               0               0              0   
4  1625       0              0               0               0              0   

   url_has_ip  url_isshorted  url_len  url_entropy  ...  pdomain_count_hyphen  \
0           0              0       15     3.640224  ...                   0.0   
1           0              0       12     2.918296  ...                   0.0   
2           0              0       12     2.918296  ...                   0.0   
3           0              0       15     3.640224  ...                   0.0   
4           0              0       29     3.935399  ...                   0.0   

   pdomain_count_atrate  p

In [8]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
params = {
    'objective': 'binary:logistic',  # For binary classification
    'max_depth': 3,
    'learning_rate': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'logloss',
    'seed': 42
}
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# Evaluate the model
y_pred = xgb_model.predict(dtest)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9958368026644463
ROC-AUC Score: 0.9906015037593985
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      2805
           1       1.00      0.98      0.99       798

    accuracy                           1.00      3603
   macro avg       1.00      0.99      0.99      3603
weighted avg       1.00      1.00      1.00      3603

